In [4]:
# Using nvcr.io/nvidia/tritonserver:24.01-py3 (a previous version of triton which has onnxruntime backend installed (cuDNN upgrade issue in the newer one))

# # First check support for tensorrt for the onnx model
# /opt/tensorrt/bin/trtexec --onnx=/workspace/model_repository/nomic-embed-vision/1/model.onnx --verbose
# /opt/tensorrt/bin/trtexec --onnx=/workspace/model_repository/UAE-Large-V1/1/model.onnx --verbose

# # from onnx nomic-embed-vision to tensortt plan file
# /opt/tensorrt/bin/trtexec --onnx=/workspace/model_repository/nomic-embed-vision/1/model.onnx --saveEngine=/workspace/model_repository/nomic-embed-vision/1/model.plan --fp16 --memPoolSize=workspace:12000

# # now do the same for the text embedding model
# /opt/tensorrt/bin/trtexec --onnx=/workspace/model_repository/UAE-Large-V1/1/model.onnx --saveEngine=/workspace/model_repository/UAE-Large-V1/1/model.plan --fp16 --memPoolSize=workspace:12000

# # Start Triton
# # My folder is network stored, so all packages should be accesible

# /opt/tritonserver/bin/tritonserver --model-repository=/workspace/model_repository

In [5]:
# CMD
# /bin/bash -c "gdown --folder https://drive.google.com/drive/folders/1E2D2ekxGa4uQ2mu9zrURKb3f8l85fFjS -O /workspace/model_repository && /opt/tritonserver/bin/tritonserver --model-repository=/workspace/model_repository"

In [ ]:
# Test text
import requests
import numpy as np

RUNPOD_API_ENDPOINT = "https://26oe7qh881svy7-5000.proxy.runpod.net/"

payload = ["a black", "hi there", "a red ball"]
response = requests.post(f"{RUNPOD_API_ENDPOINT}/infer_text", json=payload)

response_json = response.json()
result_array = np.array(response_json['result'])
result_array.shape

for item in result_array:
    print(item)

In [7]:
# Define Runpod API Endpoint
RUNPOD_API_ENDPOINT = "https://26oe7qh881svy7-5000.proxy.runpod.net/"
import faiss
import numpy as np
import pandas as pd
import requests
import os

FAISS_TEXT_INDEX_PATH = "text-embeddings/faiss_index.bin"

In [ ]:
import faiss
import numpy as np
import pandas as pd
import requests
import os

# Define Paths
FAISS_TEXT_INDEX_PATH = "text-embeddings/faiss_index.bin"
EMBEDDINGS_CSV_PATH = "text-embeddings/embeddings.csv"

# Define Runpod API Endpoint
RUNPOD_API_ENDPOINT = "https://26oe7qh881svy7-5000.proxy.runpod.net/"

# Load CSV
df_products = pd.read_csv("D:/My Works/Challenges/Sereact/Embeddings-pipeline/dataset/textual/complete/final_products.csv")

# Generate text inputs
text_inputs = df_products.apply(
    lambda row: f"Title of Product: {row['title']}\nProduct Image Description: {row['llava_generated_image_caption']}\nProduct Category: {row['category_name']}", 
    axis=1
).tolist()

# Function to get embeddings
def get_embeddings(text_inputs, batch_size=128):
    embeddings = []
    for i in range(0, len(text_inputs), batch_size):
        batch = text_inputs[i:i + batch_size]
        response = requests.post(f"{RUNPOD_API_ENDPOINT}/infer_text", json=batch)
        response_json = response.json()
        batch_embeddings = np.array(response_json['result'])  # Shape (batch_size, tokens, 1024)
        batch_embeddings = np.mean(batch_embeddings, axis=1)  # Averaging tokens
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

# Check if FAISS index exists
if os.path.exists(FAISS_TEXT_INDEX_PATH):
    print("🔹 Loading existing FAISS index...")
    index = faiss.read_index(FAISS_TEXT_INDEX_PATH)
    embeddings = np.loadtxt(EMBEDDINGS_CSV_PATH, delimiter=",")  # Load saved embeddings
else:
    print("🔹 Generating embeddings and saving FAISS index...")
    embeddings = get_embeddings(text_inputs)

    # Create FAISS Index
    d = embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(embeddings)

    # Save FAISS index
    faiss.write_index(index, FAISS_TEXT_INDEX_PATH)
    np.savetxt(EMBEDDINGS_CSV_PATH, embeddings, delimiter=",")  # Save embeddings

print(f"✅ FAISS index contains {index.ntotal} vectors.")


🔹 Generating embeddings and saving FAISS index...
✅ FAISS index contains 10200 vectors.


In [12]:
# Function to Search for Similar Products
def find_similar(query_text, index, df_products, k=5):
    response = requests.post(f"{RUNPOD_API_ENDPOINT}/infer_text", json=[query_text])
    query_embedding = np.array(response.json()['result'])
    query_embedding = np.mean(query_embedding, axis=1)  # Average token embeddings

    distances, indices = index.search(query_embedding, k)  # Search FAISS index

    # Retrieve the top similar products
    similar_products = df_products.iloc[indices[0]].copy()
    similar_products['distance'] = distances[0]  # Attach similarity scores
    
    return similar_products

# Example Query
query_text = "Macbook pro"
index = faiss.read_index(FAISS_TEXT_INDEX_PATH)
df_products = pd.read_csv("D:/My Works/Challenges/Sereact/Embeddings-pipeline/dataset/textual/complete/final_products.csv")
top_results = find_similar(query_text, index, df_products)

# Display Results
print("\n🔍 Top 5 Similar Products:\n")
print(top_results[['title', 'category_name', 'distance']])  # Show key details


🔍 Top 5 Similar Products:

                                                 title       category_name  \
244  for MacBook Pro 13 inch Case M2/M1 A2338 A2289...  Laptop Accessories   
528  for MacBook Pro 14 Inch 2023 2022 2021 Case M2...  Laptop Accessories   
636  IBENZER Compatible MacBook Pro 14 Inch Case 20...  Laptop Accessories   
548  A1398 Macbook Pro Battery A1417 for Macbook pr...  Laptop Accessories   
330  Compatible with MacBook Pro 13 inch Case M2 20...  Laptop Accessories   

       distance  
244  161.664963  
528  162.194870  
636  162.937637  
548  163.861755  
330  166.678452  


In [ ]:
# Image embeddings inference

In [14]:
import faiss
import numpy as np
import pandas as pd
import requests
import os
from io import BytesIO

# Define Paths for Image FAISS Index
FAISS_INDEX_IMG_PATH = "image_embeddings/faiss_index_images.bin"
IMAGE_EMBEDDINGS_CSV_PATH = "image_embeddings/image_embeddings.csv"

# Define Runpod API Endpoint
RUNPOD_API_ENDPOINT = "https://26oe7qh881svy7-5000.proxy.runpod.net/"

# Load CSV
df_products = pd.read_csv("D:/My Works/Challenges/Sereact/Embeddings-pipeline/dataset/textual/complete/final_products.csv")

# Extract Image URLs
image_urls = df_products['imgUrl'].tolist()

# Function to Download Images and Get Embeddings
def get_image_embeddings(image_urls, batch_size=64):
    embeddings = []
    
    for i in range(0, len(image_urls), batch_size):
        batch_urls = image_urls[i:i + batch_size]
        files = []

        # Download images
        for j, url in enumerate(batch_urls):
            response = requests.get(url)
            if response.status_code == 200:
                image_bytes = BytesIO(response.content)
                files.append(("images", (f"image_{i+j}.jpg", image_bytes, "image/jpeg")))

        # Send request to API
        response = requests.post(f"{RUNPOD_API_ENDPOINT}/infer_image", files=files)
        response_json = response.json()
        batch_embeddings = np.array(response_json['image_embeddings'])  # (batch_size, 768)
        embeddings.append(batch_embeddings)

    return np.vstack(embeddings)  # Combine all batches

# Check if FAISS index for images exists
if os.path.exists(FAISS_INDEX_IMG_PATH) and os.path.exists(IMAGE_EMBEDDINGS_CSV_PATH):
    print("🔹 Loading existing FAISS index for images...")
    index_images = faiss.read_index(FAISS_INDEX_IMG_PATH)
    image_embeddings = np.loadtxt(IMAGE_EMBEDDINGS_CSV_PATH, delimiter=",")
else:
    print("🔹 Generating image embeddings and saving FAISS index...")
    image_embeddings = get_image_embeddings(image_urls)

    # Create FAISS Index for images
    d_img = image_embeddings.shape[1]  # Should be 768 dimensions
    index_images = faiss.IndexFlatL2(d_img)
    index_images.add(image_embeddings)

    # Save FAISS index
    faiss.write_index(index_images, FAISS_INDEX_IMG_PATH)
    np.savetxt(IMAGE_EMBEDDINGS_CSV_PATH, image_embeddings, delimiter=",")

print(f"✅ FAISS index contains {index_images.ntotal} image vectors.")

🔹 Generating image embeddings and saving FAISS index...
✅ FAISS index contains 10199 image vectors.


In [16]:
# Function to Find Similar Images by URL
def find_similar_images(image_url, k=5):
    # Download image
    response = requests.get(image_url)
    if response.status_code != 200:
        raise ValueError("Failed to download image!")

    image_bytes = BytesIO(response.content)
    files = [("images", ("query_image.jpg", image_bytes, "image/jpeg"))]

    # Get query image embedding
    response = requests.post(f"{RUNPOD_API_ENDPOINT}/infer_image", files=files)
    query_embedding = np.array(response.json()['image_embeddings'])  # (1, 768)

    # Search FAISS index
    distances, indices = index_images.search(query_embedding, k)

    # Retrieve top similar images
    similar_images = df_products.iloc[indices[0]].copy()
    similar_images['distance'] = distances[0]

    return similar_images

# Example Image Query
query_image_url = "https://m.media-amazon.com/images/I/710N2S69NvL._AC_UL320_.jpg"
top_image_results = find_similar_images(query_image_url)

# Display Results
print("\n🔍 Top 5 Similar Images:\n")
print(top_image_results[['title', 'imgUrl', 'distance']])  # Show relevant details



🔍 Top 5 Similar Images:

                                                 title  \
1    Griffin Elevator Stand for Laptops - Lift Your...   
845  200Pcs Workout Stickers Gym Inspirational Stic...   
39   Laptop Stand for Desk, Computer Stand Adjustab...   
107  Laptop Stand — 360° Swivel & Adjustable Laptop...   
46   HUANUO Adjustable Laptop Stand with 360° Rotat...   

                                                imgUrl      distance  
1    https://m.media-amazon.com/images/I/710N2S69Nv...  1.390227e-07  
845  https://m.media-amazon.com/images/I/91+XDAqMOi...  6.427319e-02  
39   https://m.media-amazon.com/images/I/71F3v-25Wm...  8.010767e-02  
107  https://m.media-amazon.com/images/I/61ysSArXif...  1.025770e-01  
46   https://m.media-amazon.com/images/I/81dj+1uUqr...  1.031212e-01  


In [ ]:
# Get Health

In [ ]:
import requests
import time

RUNPOD_API_ENDPOINT = "https://lhr3tmm415zpim-5000.proxy.runpod.net/"

def wait_for_server():
    for _ in range(10):  # Check for 10 attempts (adjust if needed)
        try:
            response = requests.get(f"{RUNPOD_API_ENDPOINT}/health")
            if response.status_code == 200 and response.json().get("status") == "ok":
                print("✅ Triton is ready!")
                return True
        except requests.exceptions.RequestException:
            pass
        
        print("⏳ Waiting for Triton server to be ready...")
        time.sleep(3)  # Wait 3 seconds before retrying
    
    print("❌ Triton server is not available after multiple attempts.")
    return False

# # ✅ Wait until Triton is ready before making requests
# if wait_for_server():
#     print("🚀 Proceed with inference!")
# else:
#     print("🚨 Cannot proceed with inference. Server is unavailable.")
wait_for_server()

✅ Triton is ready!


True